# Linear Programming - More Examples

These are examples from a DataCamp course: [Supply Chain Analytics in Python](https://app.datacamp.com/learn/courses/supply-chain-analytics-in-python)

This provides a slightly different way of coding to handle problems with a bigger scales.

In [14]:
import numpy as np
import pandas as pd
from pulp import *

## 1) Simple resource scheduling

In this exercise you are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:

- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches

Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of \\$5 and the beer \\$4.5.

The objective is to <font color=coral>maximize the profit</font> for the manufacturer.

In [6]:
model = LpProblem('Maximize_Glass_Co_Profit', LpMaximize)

## Define Decision Variables
wine = LpVariable('Wine', lowBound=0, upBound=None, cat='Integer')
beer = LpVariable('Beer', lowBound=0, upBound=None, cat='Integer')

# Define Objective Functin
model += 5*wine + 4.5*beer

# Define Constraints
model += 6*wine + 5*beer <= 60
model += 10*wine + 20*beer <= 150
model += wine <= 6

print(model)
# Solve Model
model.solve()
print(f"Produce {wine.value()} batches of wine glasses")
print(f"Produce {beer.value()} batches of beer glasses")

Maximize_Glass_Co_Profit:
MAXIMIZE
4.5*Beer + 5*Wine + 0.0
SUBJECT TO
_C1: 5 Beer + 6 Wine <= 60

_C2: 20 Beer + 10 Wine <= 150

_C3: Wine <= 6

VARIABLES
0 <= Beer Integer
0 <= Wine Integer

Produce 6 batches of wine glasses
Produce 4 batches of beer glasses


## 2) Logistic planning problem

You are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West).  
The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to fulfill the regional demand at the lowest price.

| Customer |	New York |	Atlanta |
|---|---|---|
| East	| \\$211 |\\$232|
| South	| \\$232 |\\$212|
| Midwest | \\$240	|\\$230|
| West | \\$300	|\\$280|

Our decision variables in this exercise are the number of shipments for each warehouse and customer combination. You want to minimize the total cost of shipping.

In [45]:
model = LpProblem('Minimize_Transportation_Costs', LpMinimize)

# Build the lists and the demand dictionary
warehouses = ['New York','Atlanta']
customers = ['East','South','Midwest','West']
regional_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(customers, regional_demand))

var_dict = {
    ('New York', 'East'): LpVariable('ne', lowBound=0, cat='Integer'),
    ('New York', 'South'): LpVariable('ns', lowBound=0, cat='Integer'),
    ('New York', 'Midwest'): LpVariable('nm', lowBound=0, cat='Integer'),
    ('New York', 'West'): LpVariable('nw', lowBound=0, cat='Integer'),
    ('Atlanta', 'East'): LpVariable('atle', lowBound=0, cat='Integer'),
    ('Atlanta', 'South'): LpVariable('atls', lowBound=0, cat='Integer'),
    ('Atlanta', 'Midwest'): LpVariable('atlm', lowBound=0, cat='Integer'),
    ('Atlanta', 'West'): LpVariable('atlw', lowBound=0, cat='Integer')
}
costs = {
    ('New York', 'East'): 211,
    ('New York', 'South'): 232,
    ('New York', 'Midwest'): 240,
    ('New York', 'West'): 300,
    ('Atlanta', 'East'): 232,
    ('Atlanta', 'South'): 212,
    ('Atlanta', 'Midwest'): 230,
    ('Atlanta', 'West'): 280
}

# Define Objective
model += lpSum([costs[(w, c)] * var_dict[(w, c)] for c in customers for w in warehouses])

# For each customer, sum warehouse shipments and set equal to customer deman
for c in customers:
    model += lpSum([var_dict[(w,c)] for w in warehouses]) >= demand[c]
    
print(model)
model.solve()
print(*[f"{v.name} = {v.value()}" for v in model.variables()], sep='\n')

Minimize_Transportation_Costs:
MINIMIZE
232*atle + 230*atlm + 212*atls + 280*atlw + 211*ne + 240*nm + 232*ns + 300*nw + 0
SUBJECT TO
_C1: atle + ne >= 1800

_C2: atls + ns >= 1200

_C3: atlm + nm >= 1100

_C4: atlw + nw >= 1000

VARIABLES
0 <= atle Integer
0 <= atlm Integer
0 <= atls Integer
0 <= atlw Integer
0 <= ne Integer
0 <= nm Integer
0 <= ns Integer
0 <= nw Integer

atle = 0
atlm = 1100
atls = 1200
atlw = 1000
ne = 1800
nm = 0
ns = 0
nw = 0


### Using LpVariable.dicts

In [47]:
model = LpProblem('Minimize_Transportation_Costs', LpMinimize)

# Build the lists and the demand dictionary
warehouses = ['New York','Atlanta']
customers = ['East','South','Midwest','West']
regional_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(customers, regional_demand))

transport = LpVariable.dicts('route', [(w,c) for w in warehouses for c in customers],
                             lowBound=0, cat='Integer')
# var_dict = {
#     ('New York', 'East'): LpVariable('ne', lowBound=0, cat='Integer'),
#     ('New York', 'South'): LpVariable('ns', lowBound=0, cat='Integer'),
#     ('New York', 'Midwest'): LpVariable('nm', lowBound=0, cat='Integer'),
#     ('New York', 'West'): LpVariable('nw', lowBound=0, cat='Integer'),
#     ('Atlanta', 'East'): LpVariable('atle', lowBound=0, cat='Integer'),
#     ('Atlanta', 'South'): LpVariable('atls', lowBound=0, cat='Integer'),
#     ('Atlanta', 'Midwest'): LpVariable('atlm', lowBound=0, cat='Integer'),
#     ('Atlanta', 'West'): LpVariable('atlw', lowBound=0, cat='Integer')
# }
costs = {
    ('New York', 'East'): 211,
    ('New York', 'South'): 232,
    ('New York', 'Midwest'): 240,
    ('New York', 'West'): 300,
    ('Atlanta', 'East'): 232,
    ('Atlanta', 'South'): 212,
    ('Atlanta', 'Midwest'): 230,
    ('Atlanta', 'West'): 280
}

# Define Objective
model += lpSum([costs[(w, c)] * transport[(w, c)] for c in customers for w in warehouses])

# For each customer, sum warehouse shipments and set equal to customer deman
for c in customers:
    model += lpSum([transport[(w,c)] for w in warehouses]) >= demand[c]
    
print(model)
model.solve()
print(*[f"{v.name} = {v.value()}" for v in model.variables()], sep='\n')

Minimize_Transportation_Costs:
MINIMIZE
232*route_('Atlanta',_'East') + 230*route_('Atlanta',_'Midwest') + 212*route_('Atlanta',_'South') + 280*route_('Atlanta',_'West') + 211*route_('New_York',_'East') + 240*route_('New_York',_'Midwest') + 232*route_('New_York',_'South') + 300*route_('New_York',_'West') + 0
SUBJECT TO
_C1: route_('Atlanta',_'East') + route_('New_York',_'East') >= 1800

_C2: route_('Atlanta',_'South') + route_('New_York',_'South') >= 1200

_C3: route_('Atlanta',_'Midwest') + route_('New_York',_'Midwest') >= 1100

_C4: route_('Atlanta',_'West') + route_('New_York',_'West') >= 1000

VARIABLES
0 <= route_('Atlanta',_'East') Integer
0 <= route_('Atlanta',_'Midwest') Integer
0 <= route_('Atlanta',_'South') Integer
0 <= route_('Atlanta',_'West') Integer
0 <= route_('New_York',_'East') Integer
0 <= route_('New_York',_'Midwest') Integer
0 <= route_('New_York',_'South') Integer
0 <= route_('New_York',_'West') Integer

route_('Atlanta',_'East') = 0
route_('Atlanta',_'Midwest') =